<a href="https://colab.research.google.com/github/sonu786786/Responsible_AI/blob/main/Lab_04/quest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub

In [2]:
import os
os.environ["KAGGLE_API_TOKEN"] = "KGAT_155b95a19a48a3e19bfe45f85966a4a0"


In [3]:
import kagglehub

path = kagglehub.dataset_download("ryanbadai/clothes-dataset")
print("Dataset path:", path)


100%|██████████| 1.37G/1.37G [01:04<00:00, 22.8MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/ryanbadai/clothes-dataset/versions/1


In [4]:
import os
os.listdir(path)

['Clothes_Dataset']

In [5]:
dataset_path = os.path.join(path, "Clothes_Dataset")
os.listdir(dataset_path)


['Celana_Pendek',
 'Jeans',
 'Rok',
 'Gaun',
 'Kaos',
 'Sweter',
 'Jaket',
 'Hoodie',
 'Jaket_Denim',
 'Celana_Panjang',
 'Polo',
 'Blazer',
 'Kemeja',
 'Jaket_Olahraga',
 'Mantel']

In [6]:
for item in os.listdir(dataset_path):
   print(item)

Celana_Pendek
Jeans
Rok
Gaun
Kaos
Sweter
Jaket
Hoodie
Jaket_Denim
Celana_Panjang
Polo
Blazer
Kemeja
Jaket_Olahraga
Mantel


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import os

In [8]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])


In [9]:
dataset = ImageFolder(root=dataset_path, transform=transform)

print("Classes:", dataset.classes)
print("Total Images:", len(dataset))

Classes: ['Blazer', 'Celana_Panjang', 'Celana_Pendek', 'Gaun', 'Hoodie', 'Jaket', 'Jaket_Denim', 'Jaket_Olahraga', 'Jeans', 'Kaos', 'Kemeja', 'Mantel', 'Polo', 'Rok', 'Sweter']
Total Images: 7500


In [10]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_Model, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [14]:
model = CNN_Model(num_classes=len(dataset.classes)).to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
        evaluate(model, val_loader)


In [17]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")


In [18]:
train_model(model, train_loader, val_loader, epochs=10)

Epoch [1/10], Loss: 2.2306
Validation Accuracy: 41.20%
Epoch [2/10], Loss: 1.5745
Validation Accuracy: 47.13%
Epoch [3/10], Loss: 1.1666
Validation Accuracy: 52.07%
Epoch [4/10], Loss: 0.7568
Validation Accuracy: 51.67%
Epoch [5/10], Loss: 0.3943
Validation Accuracy: 53.13%
Epoch [6/10], Loss: 0.1935
Validation Accuracy: 51.67%
Epoch [7/10], Loss: 0.1145
Validation Accuracy: 51.87%
Epoch [8/10], Loss: 0.0819
Validation Accuracy: 50.67%
Epoch [9/10], Loss: 0.0827
Validation Accuracy: 50.47%
Epoch [10/10], Loss: 0.0537
Validation Accuracy: 52.93%


In [22]:
small_model = Smaller_CNN(num_classes=len(dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(small_model.parameters(), lr=0.001)

train_model(small_model, train_loader, val_loader, epochs=10)


Epoch [1/10], Loss: 2.2522
Validation Accuracy: 40.93%
Epoch [2/10], Loss: 1.5018
Validation Accuracy: 44.67%
Epoch [3/10], Loss: 1.0074
Validation Accuracy: 47.53%
Epoch [4/10], Loss: 0.5697
Validation Accuracy: 48.47%
Epoch [5/10], Loss: 0.2528
Validation Accuracy: 48.00%
Epoch [6/10], Loss: 0.1393
Validation Accuracy: 47.60%
Epoch [7/10], Loss: 0.0831
Validation Accuracy: 47.27%
Epoch [8/10], Loss: 0.0594
Validation Accuracy: 49.93%
Epoch [9/10], Loss: 0.0519
Validation Accuracy: 47.60%
Epoch [10/10], Loss: 0.0450
Validation Accuracy: 49.33%


| Model       | Conv Layers | Best Val Accuracy | Final Training Loss | Observation                                       |
| ----------- | ----------- | ----------------- | ------------------- | ------------------------------------------------- |
| Full CNN    | 3           | 53.13%            | 0.0537              | Better accuracy, overfitting observed             |
| Reduced CNN | 2           | 49.93%            | 0.0450              | Lower accuracy, reduced feature learning capacity |


Final Conclusion

A CNN model with three convolution layers achieved a maximum validation accuracy of approximately 53.13%. However, the training loss reduced significantly to near zero, indicating overfitting.

After decreasing the number of convolution layers to two, the validation accuracy dropped to approximately 49.93%. This demonstrates that reducing the number of convolution layers decreases the model’s ability to extract complex hierarchical features from images.

The deeper CNN performed better due to increased representational capacity, while the shallower CNN had limited feature abstraction capability. Therefore, decreasing the number of convolution layers negatively impacted model performance.